In [111]:
import pandas as pd
import plotly.express as px

# Load the data
file_path = '/Users/jackson/Desktop/RCA Heatmap/2022-2023_Kindergarten_Immunization_Rates_by_School.xlsx'  # Replace with your actual file path
data = pd.read_excel(file_path, skiprows=1, dtype={'Zipcode': str})  # Set dtype for Zipcode

# Assuming your dataset already has 'Latitude' and 'Longitude' columns
# If not, adjust the column names to match your dataset

# Filter out rows where 'All' or 'Latitude' or 'Longitude' is NaN
data_filtered = data.dropna(subset=['All', 'Latitude', 'Longitude'])

# Create a new column for formatted 'All' data with a "%" symbol
data_filtered['All Vaccines'] = data_filtered['All'].apply(lambda x: f"{x}%")

# Customize hover data
hover_data = {
    'All': False,  # Hide latitude
    'Latitude': False,  # Hide latitude
    'Longitude': False,  # Hide longitude
    'All Vaccines': True  # Display formatted 'All' data with "%"
}

# Create the map with Plotly using the filtered data
fig = px.scatter_mapbox(data_filtered, lat="Latitude", lon="Longitude", 
                        hover_name="School Name",  # Replace 'School Name' with your actual school name column
                        color="All", size="All",
                        hover_data=hover_data,
                        color_continuous_scale=px.colors.sequential.RdBu, size_max=15, zoom=10)

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


<ipython-input-111-c7c44e5a5362>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [144]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display, clear_output

# Load the data
file_path = '/Users/jackson/Desktop/RCA Heatmap/2022-2023_Kindergarten_Immunization_Rates_by_School.xlsx'
data = pd.read_excel(file_path, skiprows=1, dtype={'Zipcode': str})

# Convert vaccine data to numeric and replace NaNs
vaccines = ['All', 'Polio', 'DTaP', 'MMR', 'HepB', 'Varicella', 'HepA']
for vaccine in vaccines:
    data[vaccine] = pd.to_numeric(data[vaccine], errors='coerce').fillna(0)

# Filter out rows where 'Latitude' or 'Longitude' is NaN
data_filtered = data.dropna(subset=['Latitude', 'Longitude'])

def create_map(vaccine):
    # Create the formatted Vaccine Rate column
    data_filtered['Vaccine Rate'] = data_filtered[vaccine].apply(lambda x: f"{x}%")

    # Calculate the mean latitude and longitude for centering the map
    mean_lat = data_filtered["Latitude"].mean()
    mean_lon = data_filtered["Longitude"].mean()

    # Create the map with Plotly
    fig = go.Figure(go.Scattermapbox(
        lat=data_filtered["Latitude"], 
        lon=data_filtered["Longitude"], 
        hoverinfo="text",
        text=data_filtered["School Name"] + ": " + data_filtered["Vaccine Rate"],
        marker=dict(
            color=data_filtered[vaccine], 
            size=data_filtered[vaccine], 
            sizemode='area', 
            sizeref=2.*max(data_filtered[vaccine])/(40.**2), 
            sizemin=4,
            showscale=True,  # Enable color scale
            colorbar=dict(title='Vaccination Rate')  # Color bar settings
        )
    ))

    fig.update_layout(
        mapbox=dict(
            style="carto-positron",
            zoom=11.5,
            center=dict(lat=mean_lat, lon=mean_lon)
        ),
        margin={"r":0,"t":0,"l":0,"b":0}
    )

    return fig


# Create a dropdown widget for vaccine selection
vaccine_dropdown = widgets.Dropdown(
    options=vaccines,
    value='All',
    description='Vaccine:',
)

# Function to handle the update of the map
def update_map(change):
    clear_output(wait=True)
    new_vaccine = change.new
    new_map = create_map(new_vaccine)
    display(vaccine_dropdown)
    display(new_map)

# Link the dropdown to the update function
vaccine_dropdown.observe(update_map, names='value')

# Initial display
display(vaccine_dropdown)
display(create_map(vaccine_dropdown.value))


Dropdown(description='Vaccine:', options=('All', 'Polio', 'DTaP', 'MMR', 'HepB', 'Varicella', 'HepA'), value='…